## AlexNet

AlexNet架构以大比分赢得2012年的ILSVRC竞赛，它由Alex Krizhevsky, Ilya Sutskever和Geoffery Hinton等人提出。它和LeNet架构很相似，只是比LeNet更大更深。它直接将卷积层堆叠到其它层之上，而不是在每个卷积层上堆叠池化层。我们将在这里实现AlexNet。

它的传统架构如下：输入层，卷积层，最大池化层，卷积层，最大池化层，卷积层，卷积层，卷积层，全连接层，全连接层，全连接层。

对于该神经网络架构而言，最重要的实际上并不是它的层，而是在其中会使用到的其他优化神经网络的技术。它在全连接层中会使用Dropout技术将某一层50%的特征设置为0，从而避免过拟合；它还会对卷积后进行过激活的层使用本地响应归一化(LRN)的技术来使得最强烈的激活来抑制同一位置但是在不同特性图中的神经元，这可以改进该方法的泛化。

### 1. 导入必要模块

In [1]:
import time as time
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

### 2. 引入数据集

在这里，我们直接使用tensorflow中自带的数据集。

In [2]:
# These variables are all in type of numpy.
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

11490434/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


### 3. 数据预处理

将28\*28的图片填充到32\*32的规模，以便于进行输入。将图片变为3维，以便于神经网络的训练。同时，将分类变为one-hot编码，以便于后续在神经网络训练中可以使用categorical cross-entropy损失函数。

In [3]:
train_images_32 = np.zeros((60000, 32, 32), dtype=train_images.dtype)
test_images_32 = np.zeros((10000, 32, 32), dtype=test_images.dtype)

start_row = (32 - 28) // 2
start_col = (32 - 28) // 2
for i in range(60000):
  train_images_32[i][start_row:start_row+28, start_col:start_col+28] = train_images[i]
for i in range(10000):
  test_images_32[i][start_row:start_row+28, start_col:start_col+28] = test_images[i]

train_images_32 = train_images_32.reshape((60000, 32, 32, 1)).astype('float32') / 255
test_images_32 = test_images_32.reshape((10000, 32, 32, 1)).astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

print(train_images_32.shape)
print(test_images_32.shape)
print(train_labels.shape)
print(test_labels.shape)

(60000, 32, 32, 1)
(10000, 32, 32, 1)
(60000, 10)
(10000, 10)


### 4. 搭建神经网络

In [4]:
# padding = 'same' means padding = 1,
# padding = 'valid' means padding = 0.
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)))
model.add(layers.Lambda(lambda x: tf.nn.local_response_normalization(x, depth_radius=2, bias=1.0, alpha=0.00002, beta=0.75)))
model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.Lambda(lambda x: tf.nn.local_response_normalization(x, depth_radius=2, bias=1.0, alpha=0.00002, beta=0.75)))
model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))

model.add(layers.Flatten())
model.add(layers.Dense(84, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(84, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

### 5. 编译模型

In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### 6. 训练模型

In [6]:
start_time = time.time()
model.fit(train_images_32, train_labels, epochs=5, batch_size=64, validation_split=0.2)
end_time = time.time()
print("Running time:", end_time - start_time, "seconds")

Epoch 1/5
750/750 [==============================] - 22s 9ms/step - loss: 0.4894 - accuracy: 0.8447 - val_loss: 0.0650 - val_accuracy: 0.9816
Epoch 2/5
750/750 [==============================] - 8s 10ms/step - loss: 0.1314 - accuracy: 0.9670 - val_loss: 0.0524 - val_accuracy: 0.9868
Epoch 3/5
750/750 [==============================] - 7s 9ms/step - loss: 0.0913 - accuracy: 0.9778 - val_loss: 0.0671 - val_accuracy: 0.9844
Epoch 4/5
750/750 [==============================] - 8s 10ms/step - loss: 0.0687 - accuracy: 0.9840 - val_loss: 0.0453 - val_accuracy: 0.9883
Epoch 5/5
750/750 [==============================] - 7s 10ms/step - loss: 0.0570 - accuracy: 0.9854 - val_loss: 0.0521 - val_accuracy: 0.9883
Running time: 52.06026029586792 seconds


### 7. 模型应用于测试集

In [ ]:
test_loss, test_acc = model.evaluate(test_images_32, test_labels)
print(f'Test accuracy: {test_acc}')

313/313 [==============================] - 1s 4ms/step - loss: 0.0597 - accuracy: 0.9881
Test accuracy: 0.988099992275238
